In [ ]:
from pathlib import Path

from typing import List
import numpy as np
from numpy.typing import NDArray
import matplotlib.pyplot as plt
from matplotlib.axes import Axes
from shapely.geometry import Point, Polygon, GeometryCollection
import shapely.geometry

from shapely.geometry.base import BaseGeometry
from shapely.affinity import translate
import cv2
import rasterio.features

from mothi.projects import QuPathTilingProject, MaskParameter


QP_PROJECT_NAME = Path("comp_project")

In [ ]:
# create fig with ticks
def plot_image_mask(
    image: NDArray[np.int_],
    offset: float = 0.5,
    *,
    label_dist: int = 5,
    tick_distance: int = 1,
) -> Axes:

    major_ticks_x = np.arange(0, image.shape[1], label_dist)
    minor_ticks_x = np.arange(0 + offset, image.shape[1], tick_distance)

    major_ticks_y = np.arange(0, image.shape[0], label_dist)
    minor_ticks_y = np.arange(0 + offset, image.shape[0], tick_distance)

    fig, ax = plt.subplots()
    ax.set_xticks(major_ticks_x)
    ax.set_yticks(major_ticks_y)
    ax.set_xticks(minor_ticks_x, minor=True)
    ax.set_yticks(minor_ticks_y, minor=True)
    ax.grid(which="minor")
    ax.imshow(image)
    return ax


def overlay_image(img_ax: Axes, shape: BaseGeometry) -> Axes:
    offset_shape: BaseGeometry = translate(shape, -0.5, -0.5)
    img_ax.plot(*offset_shape.exterior.xy)
    return img_ax


def compare_polys(mask: NDArray, ground_truth_shapes: list[Polygon]) -> None:
    mask = mask.copy()
    ground_truth_shapes = ground_truth_shapes.copy()
    pairs: list[tuple[Polygon, Polygon]] = []
    poly_data: list[Polygon] = [
        shapely.geometry.shape(poly[0])
        for poly in rasterio.features.shapes(mask, mask != 0)
    ]
    for poly in poly_data:
        for ref_poly in ground_truth_shapes:
            if poly.intersects(ref_poly):
                pairs.append((poly, ref_poly))
                ground_truth_shapes.remove(ref_poly)
                break
        else:
            print("append empty")
            pairs.append((poly, Polygon()))
    for poly, ref_poly in pairs:
        intersection = poly.intersection(ref_poly).area
        iou = intersection / (poly.area + ref_poly.area - intersection)
        print("IoU: ", iou)
        if isinstance(poly, GeometryCollection):
            for geom in poly.geoms:
                if isinstance(geom, Polygon):
                    plt.plot(*geom.exterior.xy)
        else:
            plt.plot(*poly.exterior.xy)
        plt.plot(*ref_poly.exterior.xy)
        plt.axis("equal")
        plt.gca().invert_yaxis()
        plt.legend(["Generated", "Ground Truth"])
        plt.show()

In [ ]:
import json
from shapely import affinity, make_valid

MOTH_DATA_PATH = Path("..", "moth", "moth_data.json")
with open(MOTH_DATA_PATH, "r") as moth_data_file:
    moth_data = json.load(moth_data_file)
moth = Polygon(
    tuple(
        (int(coordinate[0]), int(coordinate[1]))
        for coordinate in moth_data.get("coordinates")[0]
    )
)
moth = affinity.translate(moth, xoff=-130, yoff=-130)
moth = make_valid(moth).geoms[0]

# get tile from QuPath with mothi
qp = QuPathTilingProject(QP_PROJECT_NAME)
moth_mask = qp.get_tile_annotation_mask(MaskParameter(1, (0, 0)), (120, 84))
# plot tile mask
img_ax: Axes = plot_image_mask(moth_mask, label_dist=10)
# for shape in ground_truth_shapes:
img_ax = overlay_image(img_ax, moth)
plt.show()
compare_polys(moth_mask, [moth])

In [ ]:
groovy_im = cv2.imread("groovy_export/white2 [x=0,y=0,w=120,h=84].png")

groovy_im = np.array(cv2.cvtColor(groovy_im, cv2.COLOR_BGR2GRAY), dtype=np.int32)
groovy_im = np.where(groovy_im > 0, 1, 0).astype(np.int32)
groovy_ax = plot_image_mask(groovy_im, label_dist=10)
# for shape in ground_truth_shapes:
groovy_ax = overlay_image(groovy_ax, moth)
plt.show()
compare_polys(groovy_im, [moth])

In [ ]:
# compare both exports
moth_groovy_diff = np.where(moth_mask > groovy_im, 1, 0)
print("Moth but not Groovy")
plt.imshow(moth_groovy_diff)
plt.show()

groovy_moth_diff = np.where(groovy_im > moth_mask, 1, 0)
print("Groovy but not Moth")
plt.imshow(groovy_moth_diff)
plt.show()

# get percanage of pixel overlap
intersection = np.sum(np.logical_and(moth_mask, groovy_im))
union = np.sum(np.logical_or(moth_mask, groovy_im))
print("Intersection: ", intersection, "Union: ", union)
iou = intersection / union
print("IoU: ", iou)

In [ ]:
# list ground truth shapes
ground_truth_shapes: List[BaseGeometry] = [
    Point(5, 5).buffer(2),
    Polygon(((10.5, 5.5), (15.5, 5.5), (15.5, 10.5), (10.5, 10.5))),
    Polygon(((28, 6), (30, 6), (30, 8), (28, 8))),
    Point(22.75, 7.75).buffer(2.5),
]

# get tile from QuPath with mothi
qp = QuPathTilingProject(QP_PROJECT_NAME)
moth_mask_export = qp.get_tile_annotation_mask(MaskParameter(0, (0, 0)), (32, 15))
# plot tile mask
img_ax: Axes = plot_image_mask(moth_mask_export, label_dist=2, tick_distance=1)
for shape in ground_truth_shapes:
    img_ax = overlay_image(img_ax, shape)
plt.show()

In [ ]:
groovy_export = cv2.imread("groovy_export/white [x=0,y=0,w=32,h=32].png")

groovy_export = np.array(
    cv2.cvtColor(groovy_export, cv2.COLOR_BGR2GRAY), dtype=np.int32
)
groovy_ax = plot_image_mask(groovy_export[:15, 0:32], label_dist=2)
for shape in ground_truth_shapes:
    groovy_ax = overlay_image(groovy_ax, shape)
plt.show()

In [ ]:
display("Comparision Ground Truth - mothi")
compare_polys(moth_mask_export, ground_truth_shapes)
display("Comparision Ground Truth - groovy")
compare_polys(groovy_export, ground_truth_shapes)